In [2]:
import pandas as pd
df = pd.read_csv("data_preprocessed.csv", index_col=0)  

In [3]:
df_wijk = pd.read_csv("Leefbaarometer-scores wijken 2002-2022.csv")
df_wijk=df_wijk.iloc[:,:4]
df_wijk=df_wijk[(df_wijk['jaar']==2020) | (df_wijk['jaar']==2022)]
df_wijk["wk_code"]=df_wijk["wk_code"].apply(lambda x: x[2:])
df_wijk.rename(columns={'wk_naam': 'name'}, inplace=True)
df_all = []
for i in range(0, len(df_wijk)-1, 2):
    row1 = df_wijk.iloc[i]
    row2 = df_wijk.iloc[i+1]
    
    # Calculate the average of the last column values
    average_value = (row1[-1] + row2[-1]) / 2
    
    # Create a new row with the average value
    new_row = [max(row1[0], row2[0]), 2021, average_value]
    
    # Append the new row to the averaged df_wijk
    df_all.append(new_row)


C:\Users\I6240624\AppData\Local\Temp\ipykernel_14924\444341776.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  average_value = (row1[-1] + row2[-1]) / 2
C:\Users\I6240624\AppData\Local\Temp\ipykernel_14924\444341776.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row = [max(row1[0], row2[0]), 2021, average_value]


In [4]:
df_buurt = pd.read_csv("Leefbaarometer-scores buurten 2002-2022.csv")
df_buurt=df_buurt.iloc[:,:4]
df_buurt=df_buurt[(df_buurt['jaar']==2020) | (df_buurt['jaar']==2022)]
df_buurt["bu_code"]=df_buurt["bu_code"].apply(lambda x: x[2:])
df_buurt.rename(columns={'bu_naam': 'name'}, inplace=True)
for i in range(0, len(df_buurt)-1, 2):
    row1 = df_buurt.iloc[i]
    row2 = df_buurt.iloc[i+1]
    
    # Calculate the average of the last column values
    average_value = (row1[-1] + row2[-1]) / 2
    
    # Create a new row with the average value
    new_row = [max(row1[0], row2[0]), 2021, average_value]
    
    # Append the new row to the averaged df_wijk
    df_all.append(new_row)
df_all = pd.DataFrame(df_all, columns=["code", "year", "liveability_score"])
df_all["code"] = df_all["code"].astype(int)

C:\Users\I6240624\AppData\Local\Temp\ipykernel_14924\3285892393.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  average_value = (row1[-1] + row2[-1]) / 2
C:\Users\I6240624\AppData\Local\Temp\ipykernel_14924\3285892393.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row = [max(row1[0], row2[0]), 2021, average_value]


In [5]:
df=pd.merge(df, df_all, left_on="gwb_code_10",right_on="code", how='inner')

In [6]:
# Define proportion function
def calculate_proportions(df, col1, col2, col3):
    df[col1] = df[col2] / df[col3]
    return df

# Define scoring function
def scoring(row, column):
    if row[column] > df[column].mean():
        return df[column].mean() / row[column]
    else:
        return row[column] / df[column].mean()

In [7]:
calculate_proportions(df, 'prop_0-14', 'Numbers.of.people.from.0.14.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_15-24', 'Number.of.people.from.15.24.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_25-44', 'Number.of.people.from.25.44.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_45-64', 'Number.of.people.from.45.64.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_65+', 'Number.of.people....65.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_low_ed', 'n_low_educated', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_med_ed', 'n_highschool_educated', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_high_ed', 'n_highly_educated', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_recipients', 'n_income_recipients', 'Number.of.inhabitants')
df['prop_employee']=((df["Number.of.inhabitants"]-df['n_income_recipients'])*(df['p_employees']/100))/df["Number.of.inhabitants"]
df["prop_self_employed"]=1-df['prop_employee']-df['prop_recipients']
df.dropna(inplace=True)

In [8]:
# Define df_score DataFrame
df_score = pd.DataFrame()

# Apply the function to each row in the 'prop_0-14' column
df_score['score_0-14'] = df.apply(scoring, column="prop_0-14", axis=1)
df_score['score_15-24'] = df.apply(scoring, column="prop_15-24", axis=1)
df_score['score_25-44'] = df.apply(scoring, column="prop_25-44", axis=1)
df_score['score_45-64'] = df.apply(scoring, column="prop_45-64", axis=1)
df_score['score_65+'] = df.apply(scoring, column="prop_65+", axis=1)
df_score['score_age']=(df_score['score_0-14']+df_score['score_15-24']+df_score['score_25-44']+df_score['score_45-64']+df_score['score_65+'])/5

In [9]:
calculate_proportions(df, 'prop_men', 'Number.of.man', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_women', 'Number.of.women', 'Number.of.inhabitants')
df_score['score_men'] = df.apply(scoring, column="prop_men", axis=1)
df_score['score_women'] = df.apply(scoring, column="prop_women", axis=1)
df_score["score_gender"]= (df_score["score_men"]+df_score["score_women"])/2

df_score['score_household_size']=df.apply(scoring, column="Average.household.size", axis=1)
df_score['score_single_family']=df.apply(scoring, column='percent.of.single.family.housing', axis=1)
df_score['score_multi_family']=df.apply(scoring, column='percent.of.multifamily.housing', axis=1)
df_score["score_family"]= (df_score["score_single_family"]+df_score["score_multi_family"])/2


df_score['score_low_ed'] = df.apply(scoring, column="prop_low_ed", axis=1)
df_score['score_med_ed'] = df.apply(scoring, column="prop_med_ed", axis=1)
df_score['score_high_ed'] = df.apply(scoring, column="prop_high_ed", axis=1)
df_score["score_education"]= (df_score["score_low_ed"]+df_score["score_med_ed"]+df_score["score_high_ed"])/3


df_score['score_recipients'] = df.apply(scoring, column="prop_recipients", axis=1)
df_score['score_employee'] = df.apply(scoring, column="prop_employee", axis=1)
df_score['score_self_employed'] = df.apply(scoring, column="prop_self_employed", axis=1)
df_score['score_employment'] = (df_score['score_recipients']+df_score['score_employee']+df_score['score_self_employed'])/3

df_score['score_density'] = df.apply(scoring, column="Population.density", axis=1)
df_score['score_liveability'] = df['liveability_score']
df_score

,score_0-14,score_15-24,score_25-44,score_45-64,score_65+,score_age,score_men,score_women,score_gender,score_household_size,...,score_low_ed,score_med_ed,score_high_ed,score_education,score_recipients,score_employee,score_self_employed,score_employment,score_density,score_liveability
0,0.717708,0.674003,0.764889,0.827972,0.421505,0.681215,0.937644,0.935850,0.936747,0.838573,...,0.639238,0.952491,0.451299,0.681009,0.969363,0.896079,0.781143,0.882195,0.329175,4.113391
1,0.759077,0.719879,0.855686,0.994688,0.762978,0.818462,0.962498,0.967187,0.964842,0.908573,...,0.607478,0.842472,0.452468,0.634139,0.991769,0.986537,0.755196,0.911167,0.608036,3.965993
2,0.853917,0.686808,0.823990,0.930168,0.943042,0.847585,0.961591,0.962230,0.961911,0.890983,...,0.897765,0.832839,0.844211,0.858272,0.922879,0.715250,0.872537,0.836889,0.604077,4.035158
3,0.908054,0.893081,0.738671,0.759890,0.865088,0.832957,0.975085,0.976658,0.975872,0.867275,...,0.860343,0.719066,0.650739,0.743383,0.990325,0.981076,0.697051,0.889484,0.527409,4.031153
4,0.754089,0.903688,0.812209,0.761722,0.522856,0.750913,0.992053,0.991998,0.992026,0.763202,...,0.688132,0.711615,0.481011,0.626919,0.966149,0.813595,0.695244,0.824996,0.943669,3.966037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,0.851775,0.971384,0.893124,0.907946,0.952583,0.915363,0.992298,0.995622,0.993960,0.890983,...,0.578132,0.799268,0.437182,0.604861,0.974235,0.846640,0.737128,0.852668,0.995024,3.866068
958,0.748417,0.999031,0.949813,0.846566,0.496591,0.808084,0.959040,0.966287,0.962664,0.995805,...,0.603772,0.883472,0.600932,0.696059,0.976010,0.842478,0.671409,0.829966,0.977772,3.750136
959,0.987183,0.819173,0.664688,0.741769,0.742222,0.791007,0.983866,0.983628,0.983747,0.829567,...,0.941598,0.713959,0.780092,0.811883,0.960046,0.865894,0.699327,0.841755,0.528867,3.857822
960,0.908088,0.677151,0.819491,0.820152,0.802687,0.805514,0.975638,0.979116,0.977377,0.954002,...,0.606484,0.836241,0.558887,0.667204,0.991063,0.989433,0.752985,0.911160,0.485428,3.931361


In [10]:
df_score.to_csv("scores.csv")